In [15]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME, TABLE_NAME
import os

# Connect to PostgreSQL and load data
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
df = pd.read_sql(f"SELECT * FROM {TABLE_NAME}", engine)

print(f"--- 1. Initial Data Shape: {df.shape} ---")
display(df.head())
print("\n--- Check Null Values ---")
print(df.isnull().sum())

--- 1. Initial Data Shape: (305291, 8) ---


,patient_age,age_unit,sex,drug_name,drug_route,reaction,seriousness,receivedate
0,32.0,801.0,2.0,ZEPBOUND,NaN,Nephrolithiasis,1,20170523
1,32.0,801.0,2.0,ZEPBOUND,NaN,Diverticulitis,1,20170523
2,32.0,801.0,2.0,ZEPBOUND,NaN,Feeling drunk,1,20170523
3,32.0,801.0,2.0,ZEPBOUND,NaN,Illness,1,20170523
4,32.0,801.0,2.0,ZEPBOUND,NaN,Pain,1,20170523



--- Check Null Values ---
patient_age    42503
age_unit       42498
sex            35499
drug_name          0
drug_route     39621
reaction           0
seriousness        0
receivedate        0
dtype: int64


In [16]:
print("--- 2. Cleaning Seriousness ---")
print("Before:", df['seriousness'].unique())

# 1. Convert to numeric, errors='coerce' จะเปลี่ยนค่าแปลกๆ เป็น NaN
df["seriousness"] = pd.to_numeric(df["seriousness"], errors='coerce')

# 2. Fill NA: สมมติว่าถ้าไม่ระบุ (NaN) ให้ถือว่าเป็น Non-Serious (2) ไว้ก่อน
df["seriousness"] = df["seriousness"].fillna(2)

# 3. Mapping: 1 -> 1 (Serious), 2 -> 0 (Non-Serious)
df["seriousness"] = df["seriousness"].map({1: 1, 2: 0}).astype(int)

print("After (Value Counts):")
print(df['seriousness'].value_counts(dropna=False))

--- 2. Cleaning Seriousness ---
Before: [1 2]
After (Value Counts):
seriousness
0    225953
1     79338
Name: count, dtype: int64


In [17]:
print("\n--- 3. Cleaning Sex ---")
# แปลงเป็น float ก่อน map เพราะบางที database ส่งมาเป็น 1.0 หรือ 1
df["sex"] = df["sex"].astype("float").map({
    1.0: "M",
    2.0: "F",
    0.0: "Unknown"
})

# จัดการค่าที่ Map ไม่เจอ (กลายเป็น NaN) ให้เป็น Unknown
df["sex"] = df["sex"].fillna("Unknown")

print("After:")
print(df['sex'].value_counts())


--- 3. Cleaning Sex ---
After:
sex
F          207968
M           61671
Unknown     35652
Name: count, dtype: int64


In [18]:
print("\n--- 4. Cleaning & Normalizing Age ---")

def convert_to_years(row):
    age = row['patient_age']
    unit = row['age_unit']
    
    # ถ้าไม่มีข้อมูลอายุ หรือ หน่วย ให้ return NaN
    if pd.isna(age) or pd.isna(unit):
        return np.nan
    
    try:
        age = float(age)
        unit = int(unit)
        
        if unit == 801:   return age            # Year
        elif unit == 802: return age / 12       # Month -> Year
        elif unit == 803: return age / 52       # Week -> Year
        elif unit == 804: return age / 365      # Day -> Year
        elif unit == 805: return age / 8760     # Hour -> Year
        elif unit == 800: return age * 10       # Decade -> Year (เผื่อเจอ)
        else: return np.nan
    except:
        return np.nan

# Apply function
df['age_years'] = df.apply(convert_to_years, axis=1).round(2)

# Check Logic: ดูเคสตัวอย่าง (เช่น แถวที่ 2 อายุ 6 เดือน ต้องได้ 0.5 ปี)
print("Check Transformation Logic:")
display(df[['patient_age', 'age_unit', 'age_years']].head())

# Filter Outliers: เอาเฉพาะอายุ 0-120 ปี
print(f"Rows before age filter: {len(df)}")
df = df[df['age_years'].between(0, 120)]
print(f"Rows after age filter: {len(df)}")


--- 4. Cleaning & Normalizing Age ---
Check Transformation Logic:


,patient_age,age_unit,age_years
0,32.0,801.0,32.0
1,32.0,801.0,32.0
2,32.0,801.0,32.0
3,32.0,801.0,32.0
4,32.0,801.0,32.0


Rows before age filter: 305291
Rows after age filter: 262788


In [ ]:
print("\n--- 5. Cleaning Text Data (Final Version) ---")

# Mapping ตามมาตรฐาน ICH E2B (R2) Code List
route_map = {
    # Common Routes
    '048': 'Oral',          '48': 'Oral',
    '058': 'Subcutaneous',  '58': 'Subcutaneous',   # ฉีดเข้าชั้นไขมัน (เจอเยอะในยาเบาหวาน/ลดน้ำหนัก)
    '042': 'Intravenous',   '42': 'Intravenous',    # ฉีดเข้าเส้นเลือด
    '065': 'Topical',       '65': 'Topical',        # ทาภายนอก
    '061': 'Topical',       '61': 'Topical',        # บางทีใช้ 61 ด้วย (แต่ทางการคือ 65)
    
    # Codes ที่คุณเจอเพิ่มเติม
    '003': 'Cutaneous',     '3':  'Cutaneous',      # ทางผิวหนัง (คล้าย Topical)
    '023': 'Intradermal',   '23': 'Intradermal',    # ฉีดเข้าชั้นผิวหนัง
    '030': 'Intramuscular', '30': 'Intramuscular',  # ฉีดเข้ากล้ามเนื้อ (IM)
    '047': 'Ophthalmic',    '47': 'Ophthalmic',     # ทางตา (ยาหยอดตา)
    '051': 'Parenteral',    '51': 'Parenteral',     # ทางการฉีด (แบบไม่ระบุชัดว่าเข้าตรงไหน)
    '059': 'Subdermal',     '59': 'Subdermal',      # ฝังใต้ผิวหนัง
    '060': 'Sublingual',    '60': 'Sublingual',     # อมใต้ลิ้น
    '062': 'Transdermal',   '62': 'Transdermal',    # แปะผิวหนัง (เช่น แผ่นแปะคุมกำเนิด)
    '064': 'Transplacental','64': 'Transplacental', # ผ่านรก (จากแม่สู่ลูก)
    
    # อื่นๆ ที่อาจเจอ
    '050': 'Rectal',        '50': 'Rectal',         # ทวารหนัก
    '054': 'Rectal',        '54': 'Rectal',         # บางทีใช้ 54
    '041': 'Inhalation',    '41': 'Inhalation',     # สูดดม
    '001': 'Auricular',     '1':  'Auricular',      # ทางหู
}

def clean_drug_route(val):
    if pd.isna(val) or str(val).strip() == "":
        return "Unknown"
    
    # 1. Clean format: เอา 'Code_' และ '.0' ออก
    val_str = str(val).strip().replace("Code_", "")
    if val_str.endswith('.0'):
        val_str = val_str[:-2]
        
    # 2. Map ข้อมูล
    if val_str in route_map:
        return route_map[val_str]
    
    # 3. Fallback: ถ้ายังไม่เจอ ให้คืนค่าเดิมแบบ Title Case
    return val_str.title() if not val_str.isdigit() else f"Unknown_Code_{val_str}"

# Apply Function
df["drug_route"] = df["drug_route"].apply(clean_drug_route)

# เช็คผลลัพธ์
print("Drug Route Sample (Final):")
print(df['drug_route'].unique())

# --- Reaction (เหมือนเดิม) ---
df["reaction"] = df["reaction"].astype(str).str.title().str.strip()


--- 5. Cleaning Text Data (Final Version) ---
Drug Route Sample (Final):
['Unknown' 'Subcutaneous' 'Topical' 'Rectal' 'Intramuscular' 'Transdermal'
 'Parenteral' 'Oral' 'Subdermal' 'Transplacental' 'Intravenous'
 'Sublingual' 'Intradermal' 'Cutaneous' 'Ophthalmic' 'Inhalation'
 'Intraperitoneal']


,patient_age,age_unit,sex,drug_name,drug_route,reaction,seriousness,receivedate,age_years
0,32.0,801.0,F,ZEPBOUND,Unknown,Nephrolithiasis,1,20170523,32.0
1,32.0,801.0,F,ZEPBOUND,Unknown,Diverticulitis,1,20170523,32.0
2,32.0,801.0,F,ZEPBOUND,Unknown,Feeling Drunk,1,20170523,32.0
3,32.0,801.0,F,ZEPBOUND,Unknown,Illness,1,20170523,32.0
4,32.0,801.0,F,ZEPBOUND,Unknown,Pain,1,20170523,32.0
...,...,...,...,...,...,...,...,...,...
305284,60.0,801.0,M,TIRZEPATIDE,Subcutaneous,Renal Tubular Injury,1,20250930,60.0
305285,60.0,801.0,M,TIRZEPATIDE,Subcutaneous,Dehydration,1,20250930,60.0
305286,60.0,801.0,M,TIRZEPATIDE,Unknown,Renal Tubular Injury,1,20250930,60.0
305287,60.0,801.0,M,TIRZEPATIDE,Unknown,Dehydration,1,20250930,60.0


In [28]:
print("\n--- 6. Cleaning Dates ---")
# errors='coerce' สำคัญมาก ถ้าเจอ 'InvalidDate' จะกลายเป็น NaT (Null) แทนที่จะ Error
df["receivedate"] = pd.to_datetime(df["receivedate"], format="%Y%m%d", errors="coerce")

print(df['receivedate'].head())
print("Null Dates:", df['receivedate'].isnull().sum())


--- 6. Cleaning Dates ---
0   2017-05-23
1   2017-05-23
2   2017-05-23
3   2017-05-23
4   2017-05-23
Name: receivedate, dtype: datetime64[ns]
Null Dates: 0


In [33]:
# Cell 7: Final Validations & Export to CSV/SQL

import os # import เผื่อไว้สำหรับการสร้าง folder

print("\n--- 7. Final Check & Export ---")

# ---------------------------------------------------------
# 1. Final Validation (ตรวจสอบครั้งสุดท้าย)
# ---------------------------------------------------------

# Drop แถวที่ข้อมูลสำคัญหายไปจริงๆ
# - reaction: ถ้าไม่รู้อาการ ก็วิเคราะห์ไม่ได้
# - seriousness: ถ้าไม่รู้ความรุนแรง ก็ทำ Prediction ไม่ได้
# - age_years: ถ้าไม่รู้อายุ (และหน่วยที่ถูกต้อง) ก็จัดกลุ่มไม่ได้
# - receivedate: ต้องมีวันที่เพื่อดู Trend
critical_cols = ["reaction", "seriousness", "age_years", "receivedate"]
df_clean = df.dropna(subset=critical_cols).copy() # ใช้ .copy() เพื่อความปลอดภัย

# Reorder Columns: จัดเรียงให้สวยงาม อ่านง่าย
final_cols_order = [
    'receivedate', 
    'drug_name', 
    'drug_route', 
    'sex', 
    'age_years', 
    'reaction', 
    'seriousness'
]

# (เผื่อกรณีคอลัมน์ไหนหายไป จะได้ไม่ Error)
available_cols = [c for c in final_cols_order if c in df_clean.columns]
df_clean = df_clean[available_cols]

print(f"Original Raw Shape: {df.shape}")
print(f"Final Clean Shape:  {df_clean.shape}")
print(f"Dropped Rows:       {df.shape[0] - df_clean.shape[0]}")

print("\n--- Sample of Final Data ---")
display(df_clean.head())

# ---------------------------------------------------------
# 2. Save to CSV (Local Backup)
# ---------------------------------------------------------
# สร้างโฟลเดอร์ปลายทางก่อน (ถ้ายังไม่มี) กัน Error
OUTPUT_DIR = "processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_CSV_PATH = f"{OUTPUT_DIR}/glp1_clean_final.csv"
df_clean.to_csv(OUTPUT_CSV_PATH, index=False)
print(f"\n✅ Saved local CSV -> {OUTPUT_CSV_PATH}")

# ---------------------------------------------------------
# 3. Save back to PostgreSQL (Production Ready)
# ---------------------------------------------------------
TARGET_TABLE = "glp1_clean"  # ตั้งชื่อตารางใหม่สำหรับข้อมูลที่ Clean แล้ว

try:
    print(f"Saving to PostgreSQL table '{TARGET_TABLE}'...")
    
    # ใช้ engine ตัวเดิมที่คุณสร้างไว้ตอนต้น (จาก Cell 1)
    # if_exists='replace': ถ้ามีตารางเก่าอยู่ ให้ลบแล้วสร้างใหม่
    # if_exists='append':  ถ้าจะเติมข้อมูลเพิ่ม (ระวังข้อมูลซ้ำ)
    df_clean.to_sql(TARGET_TABLE, engine, if_exists='replace', index=False)
    
    print(f"✅ Saved cleaned data to DB successfully!")
    
    # (Optional) Verify: ลองนับจำนวนแถวใน DB ดูว่าเข้าครบไหม
    with engine.connect() as conn:
        from sqlalchemy import text
        result = conn.execute(text(f"SELECT COUNT(*) FROM {TARGET_TABLE}"))
        count = result.scalar()
        print(f"   Verified row count in DB: {count}")
        
except Exception as e:
    print(f"❌ Error saving to database: {e}")


--- 7. Final Check & Export ---
Original Raw Shape: (262788, 9)
Final Clean Shape:  (262788, 7)
Dropped Rows:       0

--- Sample of Final Data ---


,receivedate,drug_name,drug_route,sex,age_years,reaction,seriousness
0,2017-05-23,ZEPBOUND,Unknown,F,32.0,Nephrolithiasis,1
1,2017-05-23,ZEPBOUND,Unknown,F,32.0,Diverticulitis,1
2,2017-05-23,ZEPBOUND,Unknown,F,32.0,Feeling Drunk,1
3,2017-05-23,ZEPBOUND,Unknown,F,32.0,Illness,1
4,2017-05-23,ZEPBOUND,Unknown,F,32.0,Pain,1



✅ Saved local CSV -> processed/glp1_clean_final.csv
Saving to PostgreSQL table 'glp1_clean'...
✅ Saved cleaned data to DB successfully!
   Verified row count in DB: 262788
